# ChIP-seq and CUT&RUN (single-task) model performance across different peak subsets

In [1]:
import os
os.chdir('/home/katie/bp_repo/reports/')

from util import import_peak_table

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import matplotlib.font_manager as font_manager
# from IPython.display import display
# import tqdm
# tqdm.tqdm_notebook()

import sys
sys.path.append('/home/katie/bp_repo/multitask_profile_model_SPI_GATA/')

import profile_performance

import all_functions
from all_functions import *

import plotting_helper
from plotting_helper import *

from profile_models import ModelLoader

columns = ['Test profile NLL',
           'Test profile cross entropy',
           'Test profile JSD',
           'Test profile Pearson',
           'Test profile Spearman',
           'Test profile MSE',
           'Test count Pearson',
           'Test count Spearman',
           'Test count MSE']

In [3]:
# Save preds using the CTCF ChIP-seq model!!
# YOU HAVE TO DO DATALOADING SEPARATELY BECAUSE THE TASKS_PATH WILL GET UPDATED (or ig you could update tasks_path in between uses...)

# Save predictions using model and BOTH assays' peaks
# note to self: using the DataLoader function ALREADY GETS RID OF CHRM AND CHRY!!

model = ModelLoader('/home/katie/bp_repo/pipeline_outputs/ctcf_chipseq_feb7/' + 'model.state_dict', controls=True, num_tasks=1).load_model()
path = '/home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/'

chip_unique_dataloader = DataLoader(tasks=['CTCF'], assay='chip-seq', controls=True, 
                             tasks_path='/home/katie/bp_repo/research/data/chip-seq/', subset=['full'], 
                             jitter=False, fake_controls=True,
                             premade_tsv_path=f'{path}CTCF_chip_unique_no_cut').make_loaders()['full']

chip_shared_dataloader = DataLoader(tasks=['CTCF'], assay='chip-seq', controls=True, 
                             tasks_path='/home/katie/bp_repo/research/data/chip-seq/', subset=['full'], 
                             jitter=False, fake_controls=True,
                             premade_tsv_path=f'{path}CTCF_chip_unique_shared_cut').make_loaders()['full']

cut_unique_dataloader = DataLoader(tasks=['CTCF'], assay='cutnrun', controls=True, 
                             tasks_path='/home/katie/bp_repo/research/data/cutnrun/', subset=['full'], 
                             jitter=False, fake_controls=True,
                             premade_tsv_path=f'{path}CTCF_cut_unique_no_chip').make_loaders()['full']

cut_shared_dataloader = DataLoader(tasks=['CTCF'], assay='cutnrun', controls=True, 
                             tasks_path='/home/katie/bp_repo/research/data/cutnrun/', subset=['full'], 
                             jitter=False, fake_controls=True,
                             premade_tsv_path=f'{path}CTCF_cut_unique_shared_chip').make_loaders()['full']

CTCF
/home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/CTCF_chip_unique_no_cut
(4980, 14)
Index(['chrom', 'peak_start', 'peak_end', 'name', 'score', 'strand', 'signal',
       'pval', 'qval', 'summit_offset', 'i0', 'start', 'end', 'task'],
      dtype='object')
CTCF
/home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/CTCF_chip_unique_shared_cut
(30821, 14)
Index(['chrom', 'peak_start', 'peak_end', 'name', 'score', 'strand', 'signal',
       'pval', 'qval', 'summit_offset', 'i0', 'start', 'end', 'task'],
      dtype='object')
CTCF
/home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/CTCF_cut_unique_no_chip
(24246, 23)
Index(['chrom', 'peak_start', 'peak_end', 'name', 'score', 'strand', 'signal',
       'pval', 'qval', 'summit_offset', 'i0', 'i1', 'i2', 'i3', 'i4', 'i5',
       'i6', 'i7', 'i8', 'i9', 'start', 'end', 'task'],
      dtype='object')
CTCF
/home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/CTCF

In [14]:
loaders = {'chip_unique_dataloader':chip_unique_dataloader, 
           'chip_shared_dataloader':chip_shared_dataloader, 
           'cut_unique_dataloader':cut_unique_dataloader, 
           'cut_shared_dataloader':cut_shared_dataloader}

ALL_METRICS_CHIP_MODEL = {}

for loader_key in loaders.keys():
    true_profs, log_pred_profs, true_counts, log_pred_counts = get_predictions(loaders[loader_key], model)

    metrics = profile_performance.compute_performance_metrics(
        true_profs, log_pred_profs, true_counts, log_pred_counts,
        prof_smooth_kernel_sigma=7, prof_smooth_kernel_width=81,
        #statuses=statuses, model_task_statuses=model_task_statuses,
        print_updates=False
    )
    ALL_METRICS_CHIP_MODEL[loader_key] = profile_performance.log_performance_metrics(metrics)

  0%|          | 0/39 [00:00<?, ?it/s]

	Test profile NLL: 90.657455
	Test profile cross entropy: 7.248408
	Test profile JSD: 0.412152
	Test profile Pearson: 0.101872
	Test profile Spearman: 0.182212
	Test profile MSE: 0.000009
	Test count Pearson: 0.026655
	Test count Spearman: 0.008204
	Test count MSE: 0.623768


  0%|          | 0/241 [00:00<?, ?it/s]

	Test profile NLL: 612.174503
	Test profile cross entropy: 7.317959
	Test profile JSD: 0.335644
	Test profile Pearson: 0.138051
	Test profile Spearman: 0.270613
	Test profile MSE: 0.000007
	Test count Pearson: 0.052678
	Test count Spearman: 0.089683
	Test count MSE: 4.289722


  0%|          | 0/190 [00:00<?, ?it/s]

	Test profile NLL: 197.189105
	Test profile cross entropy: 7.013790
	Test profile JSD: 0.319499
	Test profile Pearson: 0.083513
	Test profile Spearman: 0.144094
	Test profile MSE: 0.000005
	Test count Pearson: 0.133580
	Test count Spearman: 0.130444
	Test count MSE: 1.118607


  0%|          | 0/235 [00:00<?, ?it/s]

	Test profile NLL: 612.746263
	Test profile cross entropy: 7.318298
	Test profile JSD: 0.335460
	Test profile Pearson: 0.136832
	Test profile Spearman: 0.265218
	Test profile MSE: 0.000007
	Test count Pearson: 0.069797
	Test count Spearman: 0.097347
	Test count MSE: 4.299773


In [26]:
pd.DataFrame(ALL_METRICS_CHIP_MODEL.values(), columns = columns, index=ALL_METRICS_CHIP_MODEL.keys()).to_csv('/home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/chip_model_peak_subsets_metrics.csv')
                                                                                                                
                                                                                                             
                                                                                                             

In [27]:
# Save preds using the CTCF CUT&RUN model!!
# YOU HAVE TO DO DATALOADING SEPARATELY BECAUSE THE TASKS_PATH WILL GET UPDATED (or ig you could update tasks_path in between uses...)

# Save predictions using model and BOTH assays' peaks
# note to self: using the DataLoader function ALREADY GETS RID OF CHRM AND CHRY!!

model = ModelLoader('/home/katie/bp_repo/pipeline_outputs/ctcf_cutnrun_feb7/' + 'model.state_dict', controls=False, num_tasks=1).load_model()
path = '/home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/'

chip_unique_dataloader = DataLoader(tasks=['CTCF'], assay='chip-seq', controls=False, 
                             tasks_path='/home/katie/bp_repo/research/data/chip-seq/', subset=['full'], 
                             jitter=False,
                             premade_tsv_path=f'{path}CTCF_chip_unique_no_cut').make_loaders()['full']

chip_shared_dataloader = DataLoader(tasks=['CTCF'], assay='chip-seq', controls=False, 
                             tasks_path='/home/katie/bp_repo/research/data/chip-seq/', subset=['full'], 
                             jitter=False,
                             premade_tsv_path=f'{path}CTCF_chip_unique_shared_cut').make_loaders()['full']

cut_unique_dataloader = DataLoader(tasks=['CTCF'], assay='cutnrun', controls=False, 
                             tasks_path='/home/katie/bp_repo/research/data/cutnrun/', subset=['full'], 
                             jitter=False,
                             premade_tsv_path=f'{path}CTCF_cut_unique_no_chip').make_loaders()['full']

cut_shared_dataloader = DataLoader(tasks=['CTCF'], assay='cutnrun', controls=False, 
                             tasks_path='/home/katie/bp_repo/research/data/cutnrun/', subset=['full'], 
                             jitter=False,
                             premade_tsv_path=f'{path}CTCF_cut_unique_shared_chip').make_loaders()['full']


CTCF
/home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/CTCF_chip_unique_no_cut
(4980, 14)
Index(['chrom', 'peak_start', 'peak_end', 'name', 'score', 'strand', 'signal',
       'pval', 'qval', 'summit_offset', 'i0', 'start', 'end', 'task'],
      dtype='object')
CTCF
/home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/CTCF_chip_unique_shared_cut
(30821, 14)
Index(['chrom', 'peak_start', 'peak_end', 'name', 'score', 'strand', 'signal',
       'pval', 'qval', 'summit_offset', 'i0', 'start', 'end', 'task'],
      dtype='object')
CTCF
/home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/CTCF_cut_unique_no_chip
(24246, 23)
Index(['chrom', 'peak_start', 'peak_end', 'name', 'score', 'strand', 'signal',
       'pval', 'qval', 'summit_offset', 'i0', 'i1', 'i2', 'i3', 'i4', 'i5',
       'i6', 'i7', 'i8', 'i9', 'start', 'end', 'task'],
      dtype='object')
CTCF
/home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/TASK_1/CTCF

In [28]:
loaders = {'chip_unique_dataloader':chip_unique_dataloader, 
           'chip_shared_dataloader':chip_shared_dataloader, 
           'cut_unique_dataloader':cut_unique_dataloader, 
           'cut_shared_dataloader':cut_shared_dataloader}

ALL_METRICS_CUT_MODEL = {}

for loader_key in loaders.keys():
    true_profs, log_pred_profs, true_counts, log_pred_counts = get_predictions(loaders[loader_key], model)

    metrics = profile_performance.compute_performance_metrics(
        true_profs, log_pred_profs, true_counts, log_pred_counts,
        prof_smooth_kernel_sigma=7, prof_smooth_kernel_width=81,
        #statuses=statuses, model_task_statuses=model_task_statuses,
        print_updates=False
    )
    ALL_METRICS_CUT_MODEL[loader_key] = profile_performance.log_performance_metrics(metrics)

  0%|          | 0/39 [00:00<?, ?it/s]

/home/katie/bp_repo/multitask_profile_model_SPI_GATA/profile_performance.py:41: RuntimeWarning: invalid value encountered in true_divide
  cross_ent = (-log_prob_pows_sum) / trials


	Test profile NLL: 60.245127
	Test profile cross entropy: 5.628857
	Test profile JSD: 0.422120
	Test profile Pearson: 0.174455
	Test profile Spearman: 0.167835
	Test profile MSE: 0.000015
	Test count Pearson: 0.014014
	Test count Spearman: 0.031141
	Test count MSE: 0.706356


  0%|          | 0/241 [00:00<?, ?it/s]

	Test profile NLL: 248.510276
	Test profile cross entropy: 5.001862
	Test profile JSD: 0.298258
	Test profile Pearson: 0.326661
	Test profile Spearman: 0.342183
	Test profile MSE: 0.000015
	Test count Pearson: 0.121336
	Test count Spearman: 0.119872
	Test count MSE: 3.647091


  0%|          | 0/190 [00:00<?, ?it/s]

	Test profile NLL: 123.981774
	Test profile cross entropy: 5.537891
	Test profile JSD: 0.361210
	Test profile Pearson: 0.186961
	Test profile Spearman: 0.164387
	Test profile MSE: 0.000010
	Test count Pearson: 0.187257
	Test count Spearman: 0.175690
	Test count MSE: 0.865809


  0%|          | 0/235 [00:00<?, ?it/s]

	Test profile NLL: 248.946588
	Test profile cross entropy: 5.002860
	Test profile JSD: 0.297283
	Test profile Pearson: 0.328131
	Test profile Spearman: 0.342339
	Test profile MSE: 0.000015
	Test count Pearson: 0.132487
	Test count Spearman: 0.127560
	Test count MSE: 3.653745


In [29]:
pd.DataFrame(ALL_METRICS_CUT_MODEL.values(), columns = columns, index=ALL_METRICS_CUT_MODEL.keys()).to_csv('/home/katie/bp_repo/reports/katie_notebooks/round2_tasks_mar2022/cut_model_peak_subsets_metrics.csv')
                                                                                                                
                                                                                                             
                                                                                                             

In [31]:
pd.DataFrame(ALL_METRICS_CUT_MODEL.values(), columns = columns, index=ALL_METRICS_CUT_MODEL.keys())

,Test profile NLL,Test profile cross entropy,Test profile JSD,Test profile Pearson,Test profile Spearman,Test profile MSE,Test count Pearson,Test count Spearman,Test count MSE
chip_unique_dataloader,60.245127,5.628857,0.422120,0.174455,0.167835,0.000015,0.014014,0.031141,0.706356
chip_shared_dataloader,248.510276,5.001862,0.298258,0.326661,0.342183,0.000015,0.121336,0.119872,3.647091
cut_unique_dataloader,123.981774,5.537891,0.361210,0.186961,0.164387,0.000010,0.187257,0.175690,0.865809
cut_shared_dataloader,248.946588,5.002860,0.297283,0.328131,0.342339,0.000015,0.132487,0.127560,3.653745
